In [ ]:
"ml_dataset_with_fom.csv"

In [ ]:
import pandas as pd

# Load original CST simulation data
df = pd.read_csv("ml_dataset_with_fom.csv")  # Make sure the file is in the same folder

# Filter for both frequency bands
band1 = df[(df["Frequency (GHz)"] >= 2.4) & (df["Frequency (GHz)"] <= 3.0)]
band2 = df[(df["Frequency (GHz)"] >= 5.15) & (df["Frequency (GHz)"] <= 5.6)]
group_cols = ['w', 'w1', 'l21', 'l22', 'w2']

band1["abs_S11"] = band1["S11 Magnitude"].abs()
band2["abs_S11"] = band2["S11 Magnitude"].abs()

fom1 = band1.groupby(group_cols)["abs_S11"].sum().reset_index()
fom2 = band2.groupby(group_cols)["abs_S11"].sum().reset_index()

fom_total = pd.merge(fom1, fom2, on=group_cols, suffixes=("_band1", "_band2"))
fom_total["FOM"] = fom_total["abs_S11_band1"] + fom_total["abs_S11_band2"]
ml_data = fom_total[group_cols + ["FOM"]]
ml_data.to_csv("ml_dataset_with_fom.csv", index=False)
print("FOM column added and saved to 'ml_dataset_with_fom.csv")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Load dataset
data = pd.read_csv('ml_dataset_with_fom.csv')

# Define input features and target
X = data[['l21', 'l22', 'w1', 'w2', 'w']]
y = data['FOM']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Lasso Regression
lasso_model = LassoCV(cv=5, random_state=42).fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
lasso_r2 = r2_score(y_test, y_pred_lasso)
lasso_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

# Improved ANN (MLP Regressor)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ann_model = MLPRegressor(
    hidden_layer_sizes=(50, 25),
    activation='relu',
    solver='adam',
    max_iter=2000,
    random_state=42
)

ann_model.fit(X_train_scaled, y_train)
y_pred_ann = ann_model.predict(X_test_scaled)
ann_r2 = r2_score(y_test, y_pred_ann)
ann_rmse = np.sqrt(mean_squared_error(y_test, y_pred_ann))


# k-Nearest Neighbors Regression
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
knn_r2 = r2_score(y_test, y_pred_knn)
knn_rmse = np.sqrt(mean_squared_error(y_test, y_pred_knn))

# Find Max FOM and Corresponding Dimensions
max_lasso_idx = y_test.index[y_pred_lasso.argmax()]
lasso_best_dims = X_test.loc[max_lasso_idx]

max_ann_idx = y_test.index[y_pred_ann.argmax()]
ann_best_dims = X_test.loc[max_ann_idx]

max_knn_idx = y_test.index[y_pred_knn.argmax()]
knn_best_dims = X_test.loc[max_knn_idx]

# Compute Average Best Dimensions
average_best_dims = pd.DataFrame([lasso_best_dims, ann_best_dims, knn_best_dims]).mean()

# Print Results
print(f"\nLasso R² score: {lasso_r2:.4f}")
print(f"ANN R² score: {ann_r2:.4f}")
print(f"kNN R² score: {knn_r2:.4f}")

# Print Results
print(f"\nLasso RMSE score: {lasso_rmse:.4f}")
print(f"ANN RMSE score: {ann_rmse:.4f}")
print(f"kNN RMSE score: {knn_rmse:.4f}")

print("\n Lasso Best Dimensions:\n", lasso_best_dims)
print("\n ANN Best Dimensions:\n", ann_best_dims)
print("\n kNN Best Dimensions:\n", knn_best_dims)

print("\n Averaged Best Dimensions (Consensus Recommendation):\n", average_best_dims)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


# Load dataset
data = pd.read_csv('/content/drive/MyDrive/Datasets/ml_dataset_with_fom.csv')

# Define input features and target
X = data[['l21', 'l22', 'w1', 'w2', 'w']]
y = data['FOM']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create quadratic features
poly = PolynomialFeatures(degree=2, include_bias=True)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)


# Fit model
model = LinearRegression()
model.fit(X_train_poly, y_train)


# Get feature names
feature_names = poly.get_feature_names_out(input_features=X.columns)

# Get coefficients and intercept
coefficients = model.coef_
intercept = model.intercept_

# Build the equation string
terms = [f"{intercept:.4f}"] + [
    f"{coef:.4f}*{name}" for coef, name in zip(coefficients[1:], feature_names[1:])
]

equation = " + ".join(terms)

# Print the equation
print("\nPolynomial Regression Equation:")
print(f"FOM = {equation}")



# Evaluate model
y_pred = model.predict(X_test_poly)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Model RMSE: {rmse:.4f}")
print(f"Model R²: {r2:.4f}")

# Function to predict FOM for new design points
def predict_fom(design_params):
    # Create a DataFrame with correct feature names
    columns = ['l21', 'l22', 'w1', 'w2', 'w']
    design_df = pd.DataFrame([design_params], columns=columns)
    design_poly = poly.transform(design_df)
    return model.predict(design_poly)[0]


def trust_region_optimizer(initial_design, model, poly,
                           bounds,
                           initial_radius=0.1,
                           min_radius=1e-3,
                           max_iter=50):
    current = initial_design
    radius = initial_radius
    history = []

    for i in range(max_iter):
        candidates = []
        for _ in range(100):
            noise = np.random.uniform(-radius, radius, size=current.shape)
            candidate = np.clip(current + noise,
                                [b[0] for b in bounds],
                                [b[1] for b in bounds])
            candidates.append(candidate)

        candidates = np.array(candidates)
        preds = np.array([predict_fom(c) for c in candidates])

        best_idx = np.argmax(preds)
        best_candidate = candidates[best_idx]
        best_pred_fom = preds[best_idx]
        current_pred_fom = predict_fom(current)

        improvement = best_pred_fom - current_pred_fom
        if improvement > 0:
            current = best_candidate
            radius *= 1.2
        else:
            radius *= 0.5

        history.append((current.copy(), best_pred_fom, radius))

        print(f"Step {i+1}: FOM={best_pred_fom:.4f}, Radius={radius:.4f}")

        if radius < min_radius:
            print("Converged: Trust region too small.")
            break

    return current, history

# Starting design (replace with something valid for your case)
initial_design = np.array([6.96, 6.8, 1.0, 3.5, 3.33])

# Parameter bounds (update as needed based on your domain)
bounds = [(6.3, 7.3), (6.3, 7.3), (1, 3.5), (1, 3.5), (1, 3.5)]

# Run optimization
best_design, history = trust_region_optimizer(initial_design, model, poly, bounds)

print("Best design found:", best_design)

# Calculate FOM for the best design
best_fom = predict_fom(best_design)
print(f"Predicted FOM for best design: {best_fom:.4f}")
